# Loan Data
This dataset includes customers who hace paid off their loans or not.
* Loan_id： 
* --- A unique loan number assigned to each loan customers
* Loan_status： 
* --- Whether a loan is paid off, in collection, new customer yet to payoff, or paid off after the collection efforts
* Principal：
* --- Basic principal loan amount at the origination
* Terms:
* --- Can be weekly (7 days), biweekly, and monthly payoff schedule
* Effective_date:
* --- When the loan got originated and took effects
* Due_date:
* --- Since it’s one-time payoff schedule, each loan has one single due date
* Paidoff_time:
* --- The actual time a customer pays off the loan
* Pastdue_days:
* --- How many days a loan has been past due
* Age, education, gender: 
* --- A customer’s basic demographic information

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
sns.set_style("dark")
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
loan = pd.read_csv('/kaggle/input/loandata/Loan payments data.csv')

In [ ]:
loan.head(5)

In [ ]:
loan.info()

## EDA

### Loan_ID 
* This is just the identifier for each loan by each customers, we can ignore this field and will drop it at the end of EDA process

### Loan_status 
* This field is actually the target variables we are trying to predict.

In [ ]:
loan.loan_status.unique()

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = sns.countplot(loan.loan_status)
ax.set_title("Count of Loan Status")
for p in ax.patches:
    ax.annotate(str(format(int(p.get_height()), 'd')), (p.get_x(), p.get_height()*1.01))
plt.show()

### Principal
* This is the loan amount at the origination.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))
sns.boxplot(x="loan_status", y="Principal", data=loan, hue="loan_status", ax=axs[0])
sns.distplot(loan.Principal, bins=range(300, 1000, 100), ax=axs[1], kde=True)
plt.show()

### Terms
* This is the payoff schedule, which can be in week, month etc.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16,5))
sns.countplot(loan.terms, ax=axs[0])
axs[0].set_title("Count of Terms of loan")
for p in axs[0].patches:
    axs[0].annotate(str(format(int(p.get_height()), ',d')), (p.get_x(), p.get_height()*1.01))

sns.countplot(x='terms', hue='loan_status', data=loan, ax=axs[1])
axs[1].set_title("Term count breakdown by loan_status")
for t in axs[1].patches:
    if(np.isnan(float(t.get_height()))):
        axs[1].annotate(0, (t.get_x(), 0))
    else:
        axs[1].annotate(str(format(int(t.get_height()), 'd')), (t.get_x(), t.get_height()*1.01))

axs[1].legend(loc="upper left")
plt.show()

### effective_date, due_date, paid_off_time, and past_due_days
* These data points are essentially related to the date of the loan and payment.

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = sns.countplot(x='effective_date', hue='loan_status', data=loan)
ax.set_title('Loan Date')
for t in ax.patches:
    if(np.isnan(float(t.get_height()))):
        ax.annotate(0, (t.get_x(), 0))
    else:
        ax.annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
plt.show()

* Add new column paid_off_date from the paid_off_time.

In [ ]:
loan['paid_off_date'] = pd.DatetimeIndex(loan.paid_off_time).normalize()
fig = plt.figure(figsize=(16, 6))
ax = sns.countplot(x='paid_off_date', data=loan.loc[loan.loan_status.isin(['COLLECTION_PAIDOFF', 'PAIDOFF'])], hue='loan_status')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
for t in ax.patches:
    if(np.isnan(float(t.get_height()))):
        ax.annotate(0, (t.get_x(), 0))
    else:
        ax.annotate(str(format(int(t.get_height()), 'd')), (t.get_x(), t.get_height()*1.01))

ax.legend(loc='upper right')
plt.show()

* Computer the day to pay-off the loan

In [ ]:
loan['day_to_pay'] = (pd.DatetimeIndex(loan.paid_off_time).normalize() - pd.DatetimeIndex(loan.effective_date).normalize()) / np.timedelta64(1, 'D')

fig = plt.figure(figsize=(15,5))
ax = sns.countplot(x='day_to_pay', hue='terms', data=loan)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
for t in ax.patches:
    if(np.isnan(float(t.get_height()))):
        ax.annotate('', (t.get_x(), 0))
    else:
        ax.annotate(str(format(int(t.get_height()), 'd')), (t.get_x(), t.get_height()*1.01))

plt.show()

* See the distribution of PAIDOFF

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = sns.countplot(x='day_to_pay', hue='terms', data=loan.loc[loan.loan_status == 'PAIDOFF'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
for t in ax.patches:
    if(np.isnan(float(t.get_height()))):
        ax.annotate('', (t.get_x(), 0))
    else:
        ax.annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
        
plt.show()

* These data points are the demographic information of the applicant.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(16,15))
sns.distplot(loan.age, ax=axs[0][0])
axs[0][0].set_title("Total age distribution across dataset")

sns.boxplot(x='loan_status', y='age', data=loan, ax=axs[0][1])
axs[0][1].set_title("Age distribution by loan status")

sns.countplot(x='education', data=loan, ax=axs[1][0])
axs[1][0].set_title("Education count")
for t in axs[1][0].patches:
    if(np.isnan(float(t.get_height()))):
        axs[1][0].annotate('', (t.get_x(), 0))
    else:
        axs[1][0].annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
        
sns.countplot(x='education', data=loan, hue='loan_status', ax=axs[1][1])
axs[1][1].set_title("Education by loan status")
for t in axs[1][1].patches:
    if(np.isnan(float(t.get_height()))):
        axs[1][1].annotate('', (t.get_x(), 0))
    else:
        axs[1][1].annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
axs[1][1].legend(loc='upper right')

sns.countplot(x='Gender', data=loan, ax=axs[2][0])
axs[2][0].set_title("Education of Gender")
for t in axs[2][0].patches:
    if(np.isnan(float(t.get_height()))):
        axs[2][0].annotate('', (t.get_x(), 0))
    else:
        axs[2][0].annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
 
sns.countplot(x='Gender', data=loan, hue='education', ax=axs[2][1])
axs[2][1].set_title("Education of Gender")
for t in axs[2][1].patches:
    if(np.isnan(float(t.get_height()))):
        axs[2][1].annotate('', (t.get_x(), 0))
    else:
        axs[2][1].annotate(str(format(int(t.get_height()), ',d')), (t.get_x(), t.get_height()*1.01))
 
plt.show()